# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  df_da = -3 * (a ** 2) - 1.0 / (2 * (a ** 0.5))
  df_db = 3 * cos(3*b) + 2.5 * (b ** 1.5)
  df_dc = 1 / (c ** 2)
  return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
df_da1 = (f(2 + 0.000001, 3, 4) - f(2, 3, 4)) / 0.000001
df_db1 = (f(2, 3 + 0.000001, 4) - f(2, 3, 4)) / 0.000001
df_dc1 = (f(2, 3, 4 + 0.000001) - f(2, 3, 4)) / 0.000001
numerical_grad = [df_da1, df_db1, df_dc1] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
df_da2 = (f(2.000001, 3, 4) - f(1.999999, 3, 4)) / 0.000002
df_db2 = (f(2, 3.000001, 4) - f(2, 2.999999, 4)) / 0.000002
df_dc2 = (f(2, 3, 4.000001) - f(2, 3, 3.999999)) / 0.000002
numerical_grad2 = [df_da2, df_db2, df_dc2] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  def __neg__(self):
        return self * -1

  def __sub__(self, other):
        return self + (-other)

  def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data / other.data, (self, other), '/')

        def _backward():
            self.grad += (1 / other.data) * out.grad
            other.grad -= (self.data / (other.data ** 2)) * out.grad
        out._backward = _backward

        return out

  def exp(self):
        out = Value(exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

  def log(self):
        out = Value(log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out

  def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
from math import exp, log

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data / other.data, (self, other), '/')

        def _backward():
            self.grad += (1 / other.data) * out.grad
            other.grad -= (self.data / (other.data ** 2)) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        out = Value(exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def log(self):
        out = Value(log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
    counts = [logit.exp() for logit in logits]
    denominator = Value(sum(c.data for c in counts))
    out = [c / denominator for c in counts]
    print(out[3].data)

    def _backward(): # This is the derivative calculation for the softmax function
        for i in range(len(logits)):
            logits[i].grad += out[i].data * (1 - out[i].data) * out[i].grad
            for j in range(len(logits)):
                if i != j:
                    logits[i].grad -= out[j].data * out[i].data * out[j].grad
    for o in out:
        o._backward = _backward

    return out

def negative_log_likelihood(probs, label):
    return -probs[label].log()
# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)

# Correctly print probabilities
print([p.data for p in probs])

# Change: Directly use log method on probs[3] to maintain the graph
loss = negative_log_likelihood(probs, 3) # dim 3 acts as the label for this input example
loss.backward()

# Correctly print loss data
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


0.11354961935990121
[0.04177257051535045, 0.839024507462532, 0.005653302662216329, 0.11354961935990121]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535045
OK for dim 1: expected 0.8390245074625319, yours returns 0.839024507462532
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# Initial logits
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)

# Define the softmax function
def softmax(logits):
    return torch.softmax(logits, dim=0)

# Perform forward pass
probs = softmax(logits)

# Print probabilities
print("Probabilities:", probs)

# Define the negative log likelihood loss function
def negative_log_likelihood(probs, label):
    return -torch.log(probs[label])

# Compute the loss
label = 3
loss = negative_log_likelihood(probs, label)

# Perform backward pass
loss.backward()

# Print loss data
print("Loss:", loss.item())

# Expected gradients
expected_grads = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]

# Print gradients
for i, grad in enumerate(logits.grad):
    ok = 'OK' if abs(grad.item() - expected_grads[i]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {i}: expected {expected_grads[i]}, yours returns {grad.item()}")



Probabilities: tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SoftmaxBackward0>)
Loss: 2.1755154132843018
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390244841575623
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302650898695
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864504098892212
